# Code Chapter 01

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import scipy

In [ ]:
import arviz
import pymc3

In [ ]:
µ = 0.
σ = 1.
X = scipy.stats.norm(µ, σ)
x = X.rvs(1000)

In [ ]:
scipy.stats.describe(x).mean

In [ ]:
(x < 1.96).sum() - (x < -1.97).sum()